# AdaBoost y Gradient boosting con árboles en un dataset de detección de spam

Árboles de decisión y todos los ensembles propuestos funcionan en la práctica muy bien en datos tabulares como puede ser el problema de reconocimiento de spam en mails.

Se propone pues emplear el dataset id=44 de openml de detección de Spam. Son un total de 4601 muestras con 57 características.

In [4]:
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split

## Descarga del dataset Spam
X, y = fetch_openml(data_id=44, as_frame=False, cache=True, return_X_y=True)
print(X.shape)

## Partición train/test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True, random_state=23)


(4601, 57)


/usr/local/lib/python3.10/dist-packages/sklearn/datasets/_openml.py:968: FutureWarning: The default value of `parser` will change from `'liac-arff'` to `'auto'` in 1.4. You can set `parser='auto'` to silence this warning. Therefore, an `ImportError` will be raised from 1.4 if the dataset is dense and pandas is not installed. Note that the pandas parser may return different data types. See the Notes Section in fetch_openml's API doc for details.
  warn(


## AdaBoost

Probemos el clasificador de AdaBoost

In [2]:
from sklearn.ensemble import AdaBoostClassifier

ab = AdaBoostClassifier()

acc=ab.fit(X_train, y_train).score(X_test, y_test)

print(f'Precisión: {acc:.1%}')


Precisión: 93.8%


**Ejercicio** Teniendo en cuenta la documentación [AdaBoost](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.AdaBoostClassifier.html) prueba mediante grid search a encontrar mejores parámetros.

In [6]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.tree import DecisionTreeClassifier

G = {"estimator__max_depth" : [4,6,8,10], "n_estimators": [100,200,250]}

clf=AdaBoostClassifier(estimator = DecisionTreeClassifier())

GS = GridSearchCV(clf,G, scoring='accuracy', refit=True, cv=5, verbose=10)

acc = GS.fit(X_train, y_train).score(X_test, y_test)
print(f'Precisión: {acc:.1%} con {GS.best_params_}')

Fitting 5 folds for each of 12 candidates, totalling 60 fits
[CV 1/5; 1/12] START estimator__max_depth=4, n_estimators=100...................
[CV 1/5; 1/12] END estimator__max_depth=4, n_estimators=100;, score=0.952 total time=   2.4s
[CV 2/5; 1/12] START estimator__max_depth=4, n_estimators=100...................
[CV 2/5; 1/12] END estimator__max_depth=4, n_estimators=100;, score=0.950 total time=   2.0s
[CV 3/5; 1/12] START estimator__max_depth=4, n_estimators=100...................
[CV 3/5; 1/12] END estimator__max_depth=4, n_estimators=100;, score=0.951 total time=   2.1s
[CV 4/5; 1/12] START estimator__max_depth=4, n_estimators=100...................
[CV 4/5; 1/12] END estimator__max_depth=4, n_estimators=100;, score=0.936 total time=   2.0s
[CV 5/5; 1/12] START estimator__max_depth=4, n_estimators=100...................
[CV 5/5; 1/12] END estimator__max_depth=4, n_estimators=100;, score=0.944 total time=   2.6s
[CV 1/5; 2/12] START estimator__max_depth=4, n_estimators=200........

## GradientBoostingClassifier

Probemos el clasificador Gradient Boosting

In [3]:
from sklearn.ensemble import GradientBoostingClassifier

gb = GradientBoostingClassifier()

acc=gb.fit(X_train, y_train).score(X_test, y_test)

print(f'Precisión: {acc:.1%}')


Precisión: 95.2%


Los principales parámetros del GradientBoosting son:
* n_estimators (100)
* max_depth (3)
* max_features

**Ejercicio** prueba diferentes valores (lógicos) de estos parámetros mediante GridSearch.

In [7]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.tree import DecisionTreeClassifier

clf=GradientBoostingClassifier()

G = {"max_depth" : [4,6,8,10], "max_features" : [0.25, 0.5, 1.0], "n_estimators": [100,200,250]}


GS = GridSearchCV(clf,G, scoring='accuracy', refit=True, cv=5, verbose=10)

acc = GS.fit(X_train, y_train).score(X_test, y_test)
print(f'Precisión: {acc:.1%} con {GS.best_params_}')

Fitting 5 folds for each of 36 candidates, totalling 180 fits
[CV 1/5; 1/36] START max_depth=4, max_features=0.25, n_estimators=100...........
[CV 1/5; 1/36] END max_depth=4, max_features=0.25, n_estimators=100;, score=0.946 total time=   0.6s
[CV 2/5; 1/36] START max_depth=4, max_features=0.25, n_estimators=100...........
[CV 2/5; 1/36] END max_depth=4, max_features=0.25, n_estimators=100;, score=0.944 total time=   0.7s
[CV 3/5; 1/36] START max_depth=4, max_features=0.25, n_estimators=100...........
[CV 3/5; 1/36] END max_depth=4, max_features=0.25, n_estimators=100;, score=0.959 total time=   0.7s
[CV 4/5; 1/36] START max_depth=4, max_features=0.25, n_estimators=100...........
[CV 4/5; 1/36] END max_depth=4, max_features=0.25, n_estimators=100;, score=0.939 total time=   0.8s
[CV 5/5; 1/36] START max_depth=4, max_features=0.25, n_estimators=100...........
[CV 5/5; 1/36] END max_depth=4, max_features=0.25, n_estimators=100;, score=0.954 total time=   0.7s
[CV 1/5; 2/36] START max_dep

### Acelerar la búsqueda de parámetros: RandomizedSearch

Un problema recurrente que tenemos en estos experimentos es el coste temporal de encontrar los mejores parámetros mediante GridSearch. Una alternativa que reduce dicho coste temporal es el [RandomizedSearchCV](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.RandomizedSearchCV.html)

**Ejercicio** Implementa la misma búsqueda de valores que has realizado con GridSearch pero ahora con RandomizedSearch con n_iter=10 para limitar el número de pruebas a 10.

In [10]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.tree import DecisionTreeClassifier

clf=GradientBoostingClassifier()

G = {"max_depth" : [4,6,8,10], "max_features" : [0.25, 0.5, 1.0], "n_estimators": [100,200,250]}


GS = RandomizedSearchCV(clf,G, scoring='accuracy', n_iter = 10, refit=True, cv=5, verbose=10)

acc = GS.fit(X_train, y_train).score(X_test, y_test)
print(f'Precisión: {acc:.1%} con {GS.best_params_}')

Fitting 5 folds for each of 10 candidates, totalling 50 fits
[CV 1/5; 1/10] START max_depth=10, max_features=0.5, n_estimators=250...........
[CV 1/5; 1/10] END max_depth=10, max_features=0.5, n_estimators=250;, score=0.955 total time=   5.6s
[CV 2/5; 1/10] START max_depth=10, max_features=0.5, n_estimators=250...........
[CV 2/5; 1/10] END max_depth=10, max_features=0.5, n_estimators=250;, score=0.957 total time=   6.5s
[CV 3/5; 1/10] START max_depth=10, max_features=0.5, n_estimators=250...........
[CV 3/5; 1/10] END max_depth=10, max_features=0.5, n_estimators=250;, score=0.963 total time=   5.6s
[CV 4/5; 1/10] START max_depth=10, max_features=0.5, n_estimators=250...........
[CV 4/5; 1/10] END max_depth=10, max_features=0.5, n_estimators=250;, score=0.950 total time=   6.3s
[CV 5/5; 1/10] START max_depth=10, max_features=0.5, n_estimators=250...........
[CV 5/5; 1/10] END max_depth=10, max_features=0.5, n_estimators=250;, score=0.957 total time=   5.5s
[CV 1/5; 2/10] START max_dept

### Mejorar la búsqueda de parámetros: BayesianOptimization

Optimización Bayesiana es una técnica avanzada de búsqueda de parámetros. Para poder emplearla hay que instalar la librería scikit-optimize. El uso de dicha técnica es similar a las ya empleadas con la diferencia que el rango de los parámetros hay que definirlos teniendo en cuenta además el tipo de los mismos: real, entero o categórico. Además los valores a probar no se listan explícitamente si no que se especifica un rango dentro del cual el algoritmo escogerá los valores.

La ejecución puede resultar más lenta pero se pueden explorar más parámetros, de hecho se especifica un intervalo en lugar de valores concretos.



In [8]:
!pip install scikit-optimize

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.7/107.7 kB 2.0 MB/s eta 0:00:00


In [12]:
from skopt import BayesSearchCV
from skopt.space import Real, Categorical, Integer
BS = BayesSearchCV(GradientBoostingClassifier(), G, scoring='accuracy', n_iter=10, refit=True, cv=5, verbose=10)

acc = BS.fit(X_train, y_train).score(X_test, y_test)
print(f'Precisión: {acc:.1%} con {BS.best_params_}')


Fitting 5 folds for each of 1 candidates, totalling 5 fits
[CV 1/5; 1/1] START max_depth=4, max_features=1.0, n_estimators=250.............
[CV 1/5; 1/1] END max_depth=4, max_features=1.0, n_estimators=250;, score=0.955 total time=   4.5s
[CV 2/5; 1/1] START max_depth=4, max_features=1.0, n_estimators=250.............
[CV 2/5; 1/1] END max_depth=4, max_features=1.0, n_estimators=250;, score=0.951 total time=   4.7s
[CV 3/5; 1/1] START max_depth=4, max_features=1.0, n_estimators=250.............
[CV 3/5; 1/1] END max_depth=4, max_features=1.0, n_estimators=250;, score=0.961 total time=   5.0s
[CV 4/5; 1/1] START max_depth=4, max_features=1.0, n_estimators=250.............
[CV 4/5; 1/1] END max_depth=4, max_features=1.0, n_estimators=250;, score=0.944 total time=   5.4s
[CV 5/5; 1/1] START max_depth=4, max_features=1.0, n_estimators=250.............
[CV 5/5; 1/1] END max_depth=4, max_features=1.0, n_estimators=250;, score=0.958 total time=   5.7s
Fitting 5 folds for each of 1 candidates,

In [13]:
from skopt import BayesSearchCV
from skopt.space import Real, Categorical, Integer

G = {"max_depth" : Integer(2,10), "max_features" : [0.1,1.0], "n_estimators": Integer(50,200)}

BS = BayesSearchCV(GradientBoostingClassifier(), G, scoring='accuracy', n_iter=25, refit=True, cv=5, verbose=10)

acc = BS.fit(X_train, y_train).score(X_test, y_test)
print(f'Precisión: {acc:.1%} con {GS.best_params_}')

/usr/local/lib/python3.10/dist-packages/skopt/space/space.py:110: UserWarning: Dimension [0.1, 1.0] was inferred to Real(low=0.1, high=1.0, prior='uniform', transform='identity'). In upcoming versions of scikit-optimize, it will be inferred to Categorical(categories=(0.1, 1.0), prior=None). See the documentation of the check_dimension function for the upcoming API.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/skopt/space/space.py:110: UserWarning: Dimension [0.1, 1.0] was inferred to Real(low=0.1, high=1.0, prior='uniform', transform='identity'). In upcoming versions of scikit-optimize, it will be inferred to Categorical(categories=(0.1, 1.0), prior=None). See the documentation of the check_dimension function for the upcoming API.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/skopt/space/space.py:110: UserWarning: Dimension [0.1, 1.0] was inferred to Real(low=0.1, high=1.0, prior='uniform', transform='identity'). In upcoming versions of scikit-optimize, it will b

Fitting 5 folds for each of 1 candidates, totalling 5 fits
[CV 1/5; 1/1] START max_depth=8, max_features=0.18693114344560607, n_estimators=112
[CV 1/5; 1/1] END max_depth=8, max_features=0.18693114344560607, n_estimators=112;, score=0.955 total time=   1.0s
[CV 2/5; 1/1] START max_depth=8, max_features=0.18693114344560607, n_estimators=112
[CV 2/5; 1/1] END max_depth=8, max_features=0.18693114344560607, n_estimators=112;, score=0.950 total time=   1.0s
[CV 3/5; 1/1] START max_depth=8, max_features=0.18693114344560607, n_estimators=112
[CV 3/5; 1/1] END max_depth=8, max_features=0.18693114344560607, n_estimators=112;, score=0.969 total time=   1.0s
[CV 4/5; 1/1] START max_depth=8, max_features=0.18693114344560607, n_estimators=112
[CV 4/5; 1/1] END max_depth=8, max_features=0.18693114344560607, n_estimators=112;, score=0.946 total time=   0.9s
[CV 5/5; 1/1] START max_depth=8, max_features=0.18693114344560607, n_estimators=112
[CV 5/5; 1/1] END max_depth=8, max_features=0.186931143445606

## HistGradientBoostingClassifier
Un problema del clasificador GradientBoosting implementado en sklearn es su velocidad. Es un clasificador muy lento de entrenar. Por ello sklearn propone otro tipo de algoritmo de GradientBoosting que soporta paralelismo con OMP además de otras mejoras computacionales basadas en la discretización de las componentes mediante un histograma.

Este otro algoritmo se denomina **HistGradientBoostingClassifier**. Su tiempo de ejecución es mucho mejor. Además se pueden obtener mejores resultados.



In [ ]:
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier

hgb = HistGradientBoostingClassifier()

acc=hgb.fit(X_train, y_train).score(X_test, y_test)

print(f'Precisión: {acc:.1%}')

Precisión: 95.5%


**Ejercicio** Analiza en la documentación [HistGradientBoosting](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.HistGradientBoostingClassifier.html) los parámetros más relevantes y realiza una búsqueda de parámetros para obtener el mejor clasificador. A la hora de establecer los valores de los parámetros sería interesante fijarnos en los valores por defecto que se han empleado en el ejercicio anterior y poder explorar alrededor de dichos valores por defecto para conseguir mejores resultados.



In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import HistGradientBoosting
from sklearn.tree import DecisionTreeClassifier

clf=GradientBoostingClassifier()

G = {"max_depth" : [4,6,8,10], "max_features" : [0.25, 0.5, 1.0], "n_estimators": [100,200,250]}


GS = RandomizedSearchCV(clf,G, scoring='accuracy', n_iter = 10, refit=True, cv=5, verbose=10)

acc = GS.fit(X_train, y_train).score(X_test, y_test)
print(f'Precisión: {acc:.1%} con {GS.best_params_}')